In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Fuzzy Entropy

In [2]:
def min_max_scaler(data):
    """
    Scales data to a range of -1 to 1 using min-max scaling.

    Args:
      data: A NumPy array containing the data to be scaled.

    Returns:
      A NumPy array with the same shape as the input data, scaled to -1 to 1.
    """
    min_value = np.min(data)
    max_value = np.max(data)
    scaled_data = (data - min_value) / (max_value - min_value) * 2 - 1
    return scaled_data

In [3]:
def calculate_fuzzy_entropy(data, m=2, r=None):
    """
    Calculates the fuzzy entropy of a time series data segment.

    Args:
      data: A 1D array containing the time series data.
      m: The order of the fuzzy entropy.
      r: The radius of the fuzzy set.

    Returns:
      A list of fuzzy entropy values for each data segment.
    """

    # Calculate fuzzy sets (membership degrees)
    n = len(data)
    if r is None:
        r = 0.15 * np.std(data)
    fuzzy_sets = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            # Fuzzy set membership
            fuzzy_sets[i, j] = np.exp(-abs(data[i] - data[j]) / r)

    # Calculate fuzzy entropy for each segment
    fuzzy_entropies = []
    for i in range(m, len(data)):
        # Extract segment
        fuzzy_segment = fuzzy_sets[i - m : i, i - m : i]

        # Shannon entropy
        fuzzy_entropy = -np.sum(fuzzy_segment * np.log2(fuzzy_segment))
        fuzzy_entropies.append(fuzzy_entropy)

    return fuzzy_entropies

Retrieve dataset and extract rest phase trial results

In [4]:
participant_trial_df = pd.read_csv("../data/Participant Trial Data.csv")
participant_trial_df.dropna(inplace=True)
participant_trial_df.reset_index(drop=True, inplace=True)

participant_df = pd.read_csv("../data/Participant Data.csv")

electrodes = [
    "Fp1[1]",
    "Fp2[2]",
    "F3[3]",
    "F4[4]",
    "C3[5]",
    "C4[6]",
    "P3[7]",
    "P4[8]",
    "O1[9]",
    "O2[10]",
    "F7[11]",
    "F8[12]",
    "T3[13]",
    "T4[14]",
    "T5[15]",
    "T6[16]",
    "Fz[17]",
    "Pz[18]",
    "Cz[19]",
]

In [5]:
rest_phase_trial_df = participant_trial_df[
    participant_trial_df["phase"].isin([1, 3])
]  # Phases 1 and 3 are rest phases
rest_phase_trial_df = rest_phase_trial_df[
    rest_phase_trial_df["duration"] == 5.0
]  # Extract only those with a duration of 5 seconds
rest_phase_trial_df = rest_phase_trial_df[
    rest_phase_trial_df["sample_frequency"] == 200.0
]  # Extract only those with a sample frequency of 200 Hz

rest_phase_trial_df

,id_trial_phase,id,phase,signal_length,event_marker_length,sample_frequency,duration
4,1_23_1,1,1,1000.0,50.0,200.0,5.0
6,1_23_3,1,3,1000.0,50.0,200.0,5.0
16,10_2_1,10,1,1000.0,50.0,200.0,5.0
18,10_2_3,10,3,1000.0,50.0,200.0,5.0
20,10_3_1,10,1,1000.0,50.0,200.0,5.0
...,...,...,...,...,...,...,...
235,6_3_3,6,3,1000.0,50.0,200.0,5.0
237,7_2_1,7,1,1000.0,50.0,200.0,5.0
239,7_2_3,7,3,1000.0,50.0,200.0,5.0
241,7_3_1,7,1,1000.0,50.0,200.0,5.0


In [8]:
# Dictionary to store fuzzy entropy data for each trial with category information
trial_fuzzy_entropy_data = {}

# Loop through each trial ID in the "rest_phase_trial_df"
for trial_id in rest_phase_trial_df["id_trial_phase"]:
    # Read EEG data for the current trial
    eeg_data = pd.read_csv(f"../data/signal_data/{trial_id}.csv")

    # Extract participant ID and category from respective dataframes
    participant_id = rest_phase_trial_df[
        rest_phase_trial_df["id_trial_phase"] == trial_id
    ]["id"].values[0]
    participant_category = participant_df[participant_df["id"] == participant_id][
        "category"
    ].values[0]

    # Initialize a dictionary to store fuzzy entropy values for each electrode within the current trial
    trial_fuzzy_entropy = {"category": participant_category, "fuzzy_entropy": {}}

    # Loop through all electrodes in the EEG data
    for electrode_name in eeg_data.columns:
        # Extract EEG data for the current electrode
        electrode_data = eeg_data[electrode_name].to_numpy()

        # Analyze the data in segments of 200 data points (5 segments starting from every 200th point correspoinding to 1 second)
        for segment_index in range(5):
            # Extract the current data segment based on the segment index
            data_to_fuzzy = electrode_data[
                (segment_index * 200) : ((segment_index + 1) * 200)
            ]

            # Scale the data segment to a range of -1 to 1 using min-max scaling
            scaled_data_to_fuzzy = min_max_scaler(data_to_fuzzy)

            # Calculate the fuzzy entropy for the scaled data segment
            fuzzy_entropy_value = calculate_fuzzy_entropy(scaled_data_to_fuzzy)

            # Store the calculated fuzzy entropy for the current electrode and segment
            trial_fuzzy_entropy["fuzzy_entropy"][electrode_name] = fuzzy_entropy_value

    # Add the dictionary for the current trial to the main results dictionary
    trial_fuzzy_entropy_data[trial_id] = trial_fuzzy_entropy

 Generate dataset

In [47]:
fuzzy_entropy_dataset = {"id_trial_phase": [], "category": []}

for key in trial_fuzzy_entropy_data:
    fuzzy_entropy_dataset["id_trial_phase"].append(key)
    fuzzy_entropy_dataset["category"].append(trial_fuzzy_entropy_data[key]["category"])

    for electrode in trial_fuzzy_entropy_data[key]["fuzzy_entropy"]:
        if electrode in electrodes:
            for i in range(
                len(trial_fuzzy_entropy_data[key]["fuzzy_entropy"][electrode])
            ):
                if f"{electrode}_{i}" not in fuzzy_entropy_dataset.keys():
                    fuzzy_entropy_dataset[f"{electrode}_{i}"] = [
                        trial_fuzzy_entropy_data[key]["fuzzy_entropy"][electrode][i]
                    ]
                else:
                    fuzzy_entropy_dataset[f"{electrode}_{i}"].append(
                        trial_fuzzy_entropy_data[key]["fuzzy_entropy"][electrode][i]
                    )

fuzzy_entropy_df = pd.DataFrame(fuzzy_entropy_dataset)
category_column = fuzzy_entropy_df["category"]
fuzzy_entropy_df.drop(columns=["id_trial_phase", "category"], inplace=True)
columns_means = fuzzy_entropy_df.mean()
fuzzy_entropy_df = fuzzy_entropy_df.loc[:, columns_means != 0]
fuzzy_entropy_df["category"] = category_column

SVM Model

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Separate features and labels
label = "category"
features = [column for column in fuzzy_entropy_df.columns if column != label]

X = fuzzy_entropy_df[features]  # Adjust feature names based on your data
y = fuzzy_entropy_df[label]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Define and train RBF SVC model
model = SVC(kernel="rbf", C=1.0, gamma="auto")
model.fit(X_train, y_train)

# Predict labels on test data and calculate accuracy
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.6190
